In [1]:
import numpy as np
import pandas as pd

In [2]:
def prepare_problem(xs, ys, quantile):
    X_train, y_train = xs.to_frame(), ys
    X = np.concatenate([
        np.full(X_train.shape[0], 1.0).reshape(-1, 1),
        X_train.to_numpy()
    ], axis=1)
    n, k = X.shape

    c = np.concatenate([
        np.full(2 * k, 0.0),
        quantile * np.full(n, 1.0),
        (1 - quantile) * np.full(n, 1.0)
    ])
    A = np.concatenate([
        X, -X, np.identity(n), -np.identity(n)
    ], axis=1)
    m = A.shape[1]
    b = y_train.to_numpy().reshape(-1, 1)

    return c, A, b

In [3]:
def steepest_edge_simplex(_c, _A, _b):
    # See: D. Goldfarb, J. K. Reid, "A practicable steepest-edge simplex algorithm"
    '''
    Problem:
    T(c) * x -> min
    Constraints:
    A * x = b
    x >= 0
    '''
    from numpy import linalg as la

    def get_pivot_indices(A):
        from sympy import Matrix

        return Matrix(A).rref()[1]

    def get_row(A, i):
        return np.array([A[i, :]])

    def get_column(A, i):
        return A[:, i].reshape(-1, 1)

    def e(n, i):
        return np.eye(n)[:, i].reshape(-1, 1)

    def permutation_matrix(n, i, j):
        P = np.eye(n)
        P[:, [i, j]] = P[:, [j, i]]
        return P

    c = np.array(_c)
    if len(c.shape) == 1:
        c = c.reshape(-1, 1)
    A = np.array(_A)
    m, n = A.shape
    b = np.array(_b)
    if len(b.shape) == 1:
        b = b.reshape(-1, 1)

    def is_optimal(z):
        return np.all(z[m:, :] >= 0)

    def get_pivot_column_index(z, gamma):
        max_value = -1
        q = -1
        for i in range(m, n):
            z_i = z[i, 0]
            if z_i >= 0:
                continue
            value = z_i**2 / gamma[i, 0]
            if value > max_value:
                max_value = value
                q = i
        return q

    def is_unbounded(w):
        return np.all(w <= 0)

    def get_pivot_index(x, w):
        min_value = abs(x.max()) / (abs(w.min()) + 1) + 1
        p = -1
        for i in range(m):
            w_i = w[i, 0]
            if w_i <= 0:
                continue
            value = x[i, 0] / w_i
            if value < min_value:
                min_value = value
                p = i
        return p

    def revise_x(x, w, p):
        x_p = x[p, 0] / w_p
        revised_x = x - np.block([[w * x_p], [np.zeros((n - m, 1))]])
        revised_x[p, 0] = x_p
        return revised_x

    def revise_z(z, alpha, w_p, p, q):
        revised_z = z.copy()
        for i in range(n):
            if i < m:
                continue
            if i == q:
                revised_z[i, 0] = -z[i, 0] / w_p
            else:
                revised_z[i, 0] = z[i, 0] - alpha[i, 0] * z[q, 0]
        return revised_z

    def revise_gamma(gamma, alpha, wTA, w_p, p, q):
        revised_gamma = gamma.copy()
        for i in range(n):
            if i < m:
                continue
            if i == q:
                revised_gamma[i, 0] = gamma[i, 0] / w_p**2
            else:
                revised_gamma[i, 0] = max(
                    gamma[i, 0] - 2 * alpha[i, 0] * wTA[i, 0] + alpha[i, 0]**2 * gamma[q, 0],
                    1 + alpha[i, 0]**2
                )
        return revised_gamma

    assert m < n, "Bad shape of matrix A: {0}".format(A.shape)
    assert b.shape[0] == m and b.shape[1] == 1, "Bad shape of matrix b: {0}".format(b.shape)
    assert c.shape[0] == n and c.shape[1] == 1, "Bad shape of matrix c: {0}".format(c.shape)

    pivot_indices = get_pivot_indices(A)
    assert len(pivot_indices) == m, \
        "Constraint matrix should have {0} independent columns, but have only {1}".format(m, len(pivot_indices))
    dependent_indices = tuple(set(range(n)) - set(pivot_indices))
    A1 = A[:, pivot_indices]
    A1_inv = la.inv(A1)
    x = np.block([[np.dot(A1_inv, b)], [np.zeros((n - m, 1))]])
    A2 = A[:, dependent_indices]
    A = np.block([[A1, A2]])
    c1 = c[pivot_indices, :]
    c2 = c[dependent_indices, :]
    c = np.block([[c1], [c2]])
    N = np.block([[A1, A2], [np.zeros((n - m, m)), np.eye(n - m)]])
    N_inv = np.block([[A1_inv, -np.dot(A1_inv, A2)], [np.zeros((n - m, m)), np.eye(n - m)]])
    z = np.dot(c.T, N_inv).T
    gamma = np.diag(np.dot(N_inv.T, N_inv)).reshape(-1, 1)

    iteration_number = 0

    while True:
        # (a)
        if is_optimal(z):
            indices = np.array(pivot_indices + dependent_indices).reshape(-1, 1)
            x_indexed = np.block([x, indices])
            x_sorted = x_indexed[x_indexed[:, 1].argsort()]
            return x_sorted[:, 0]
        # (b)
        q = get_pivot_column_index(z, gamma)
        w = np.dot(A1_inv, get_column(A, q))
        # (c)
        if is_unbounded(w):
            raise Exception("problem is unbounded")
            return None
        # (d)
        p = get_pivot_index(x, w)
        w_p = w[p, 0]
        # (e)
        revised_x = revise_x(x, w, p)
        # (f)
        z_q = c[q, 0] - np.dot(c1.T, w)[0, 0]
        gamma_q = 1 + np.dot(w.T, w)[0, 0]
        w = np.dot(A1_inv.T, w)
        # revise indices
        pivot_new_index_at_p = dependent_indices[q - m]
        dependent_new_index_at_q = pivot_indices[p]
        pivot_indices = tuple([(pivot_new_index_at_p if i == p else idx) for i, idx in enumerate(pivot_indices)])
        dependent_indices = tuple([(dependent_new_index_at_q if i == q - m else idx) for i, idx in enumerate(dependent_indices)])
        # revise N
        revised_N = np.dot((N + np.dot(e(n, q), (e(n, p) - e(n, q)).T)), permutation_matrix(n, p, q))
        revised_N_inv = la.inv(revised_N)
        # revise A1, A2
        revised_A1 = revised_N[:m, :m]
        revised_A1_inv = la.inv(revised_A1)
        revised_A2 = revised_N[:m, m:]
        revised_A = np.block([[revised_A1, revised_A2]])
        # y
        y = np.dot(revised_A1_inv, e(m, p)) # TODO: not used?
        # alpha
        alpha = -get_row(N_inv, p).reshape(-1, 1) # = get_row(np.dot(A1_inv, A2), p).reshape(-1, 1)
        revised_alpha = alpha / alpha[q, 0] # = get_row(np.dot(revised_A1_inv, revised_A2), p).reshape(-1, 1)
        # revise z and gamma
        revised_z = revise_z(z, revised_alpha, w_p, p, q)
        revised_gamma = revise_gamma(gamma, alpha, np.dot(w.T, A).reshape(-1, 1), w_p, p, q)

        # revision
        x = revised_x.copy()
        N = revised_N.copy()
        N_inv = revised_N_inv.copy()
        A1 = revised_A1.copy()
        A1_inv = revised_A1_inv.copy()
        A2 = revised_A2.copy()
        A = revised_A.copy()
        z = revised_z.copy()
        gamma = revised_gamma.copy()

        iteration_number += 1
        if iteration_number == 1_000_000:
            raise Exception("too long computation")
            break

c = [1, 0, 0, 0, -1]
A = pd.DataFrame({'x1': [0, 0, 0], 'x2': [0, 0, 1], 'x3': [0, 1, 0], 'x4': [1, 0, 0], 'x5': [0, 0, 1]})
b = np.array([[0], [1], [2]])
steepest_edge_simplex(c, A, b)

array([0., 0., 1., 0., 2.])

In [4]:
df = pd.DataFrame({
    "x": [0, 0, 1, 1, 1, 2, 2, 3],
    "y": [0, 1, 0, 1, 2, 1, 2, 3],
})
df.head()

,x,y
0,0,0
1,0,1
2,1,0
3,1,1
4,1,2


In [5]:
c, A, b = prepare_problem(df.x, df.y, 0.01)
print(c, A, b)
steepest_edge_simplex(c, A, b)

[0.   0.   0.   0.   0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.99 0.99
 0.99 0.99 0.99 0.99 0.99 0.99] [[ 1.  0. -1. -0.  1.  0.  0.  0.  0.  0.  0.  0. -1. -0. -0. -0. -0. -0.
  -0. -0.]
 [ 1.  0. -1. -0.  0.  1.  0.  0.  0.  0.  0.  0. -0. -1. -0. -0. -0. -0.
  -0. -0.]
 [ 1.  1. -1. -1.  0.  0.  1.  0.  0.  0.  0.  0. -0. -0. -1. -0. -0. -0.
  -0. -0.]
 [ 1.  1. -1. -1.  0.  0.  0.  1.  0.  0.  0.  0. -0. -0. -0. -1. -0. -0.
  -0. -0.]
 [ 1.  1. -1. -1.  0.  0.  0.  0.  1.  0.  0.  0. -0. -0. -0. -0. -1. -0.
  -0. -0.]
 [ 1.  2. -1. -2.  0.  0.  0.  0.  0.  1.  0.  0. -0. -0. -0. -0. -0. -1.
  -0. -0.]
 [ 1.  2. -1. -2.  0.  0.  0.  0.  0.  0.  1.  0. -0. -0. -0. -0. -0. -0.
  -1. -0.]
 [ 1.  3. -1. -3.  0.  0.  0.  0.  0.  0.  0.  1. -0. -0. -0. -0. -0. -0.
  -0. -1.]] [[0]
 [1]
 [0]
 [1]
 [2]
 [1]
 [2]
 [3]]


array([-2.,  2.,  0.,  0.,  2.,  3.,  0.,  1.,  2., -1.,  0., -1.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [6]:
c, A, b = prepare_problem(df.x, df.y, 0.25)
steepest_edge_simplex(c, A, b)

array([ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  1.00000000e+00,  0.00000000e+00,  1.11022302e-16,
        1.00000000e+00, -1.00000000e+00,  0.00000000e+00, -1.11022302e-16,
        0.00000000e+00,  0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00])

In [13]:
def simplex(_c, _A, _b):
    # See: D. Goldfarb, J. K. Reid, "A practicable steepest-edge simplex algorithm"
    '''
    Problem:
    T(c) * x -> min
    Constraints:
    A * x = b
    x >= 0
    '''
    from numpy import linalg as la

    def get_pivot_indices(A):
        from sympy import Matrix

        return Matrix(A).rref()[1]

    def get_row(A, i):
        return np.array([A[i, :]])

    def get_column(A, i):
        return A[:, i].reshape(-1, 1)

    def e(n, i):
        return np.eye(n)[:, i].reshape(-1, 1)

    def permutation_matrix(n, i, j):
        P = np.eye(n)
        P[:, [i, j]] = P[:, [j, i]]
        return P

    c = np.array(_c)
    if len(c.shape) == 1:
        c = c.reshape(-1, 1)
    A = np.array(_A)
    m, n = A.shape
    b = np.array(_b)
    if len(b.shape) == 1:
        b = b.reshape(-1, 1)

    def is_optimal(z):
        return np.all(z[m:, :] >= 0)

    def get_pivot_column_index(z):
        max_value = -1
        q = -1
        for i in range(m, n):
            z_i = z[i, 0]
            if z_i >= 0:
                continue
            value = z_i**2
            if value > max_value:
                max_value = value
                q = i
        return q

    def is_unbounded(w):
        return np.all(w <= 0)

    def get_pivot_index(x, w):
        min_value = abs(x.max()) / (abs(w.min()) + 1) + 1
        p = -1
        for i in range(m):
            w_i = w[i, 0]
            if w_i <= 0:
                continue
            value = x[i, 0] / w_i
            if value < min_value:
                min_value = value
                p = i
        return p

    def revise_x(x, w, p):
        x_p = x[p, 0] / w_p
        revised_x = x - np.block([[w * x_p], [np.zeros((n - m, 1))]])
        revised_x[p, 0] = x_p
        return revised_x

    def revise_z(z, alpha, w_p, p, q):
        revised_z = z.copy()
        for i in range(n):
            if i < m:
                continue
            if i == q:
                revised_z[i, 0] = -z[i, 0] / w_p
            else:
                revised_z[i, 0] = z[i, 0] - alpha[i, 0] * z[q, 0]
        return revised_z

    assert m < n, "Bad shape of matrix A: {0}".format(A.shape)
    assert b.shape[0] == m and b.shape[1] == 1, "Bad shape of matrix b: {0}".format(b.shape)
    assert c.shape[0] == n and c.shape[1] == 1, "Bad shape of matrix c: {0}".format(c.shape)

    pivot_indices = get_pivot_indices(A)
    assert len(pivot_indices) == m, \
        "Constraint matrix should have {0} independent columns, but have only {1}".format(m, len(pivot_indices))
    dependent_indices = tuple(set(range(n)) - set(pivot_indices))
    A1 = A[:, pivot_indices]
    A1_inv = la.inv(A1)
    x = np.block([[np.dot(A1_inv, b)], [np.zeros((n - m, 1))]])
    A2 = A[:, dependent_indices]
    A = np.block([[A1, A2]])
    c1 = c[pivot_indices, :]
    c2 = c[dependent_indices, :]
    c = np.block([[c1], [c2]])
    N = np.block([[A1, A2], [np.zeros((n - m, m)), np.eye(n - m)]])
    N_inv = np.block([[A1_inv, -np.dot(A1_inv, A2)], [np.zeros((n - m, m)), np.eye(n - m)]])
    z = np.dot(c.T, N_inv).T

    iteration_number = 0

    while True:
        # (a)
        if is_optimal(z):
            print("ITERATIONS: {0}".format(iteration_number))
            indices = np.array(pivot_indices + dependent_indices).reshape(-1, 1)
            x_indexed = np.block([x, indices])
            x_sorted = x_indexed[x_indexed[:, 1].argsort()]
            return x_sorted[:, 0]
        # (b)
        q = get_pivot_column_index(z)
        w = np.dot(A1_inv, get_column(A, q))
        # (c)
        if is_unbounded(w):
            raise Exception("problem is unbounded")
            return None
        # (d)
        p = get_pivot_index(x, w)
        w_p = w[p, 0]
        # (e)
        revised_x = revise_x(x, w, p)
        # (f)
        z_q = c[q, 0] - np.dot(c1.T, w)[0, 0]
        # revise indices
        pivot_new_index_at_p = dependent_indices[q - m]
        dependent_new_index_at_q = pivot_indices[p]
        pivot_indices = tuple([(pivot_new_index_at_p if i == p else idx) for i, idx in enumerate(pivot_indices)])
        dependent_indices = tuple([(dependent_new_index_at_q if i == q - m else idx) for i, idx in enumerate(dependent_indices)])
        # revise N
        revised_N = np.dot((N + np.dot(e(n, q), (e(n, p) - e(n, q)).T)), permutation_matrix(n, p, q))
        revised_N_inv = la.inv(revised_N)
        # revise A1, A2
        revised_A1 = revised_N[:m, :m]
        revised_A1_inv = la.inv(revised_A1)
        revised_A2 = revised_N[:m, m:]
        revised_A = np.block([[revised_A1, revised_A2]])
        # alpha
        alpha = -get_row(N_inv, p).reshape(-1, 1) # = get_row(np.dot(A1_inv, A2), p).reshape(-1, 1)
        revised_alpha = alpha / alpha[q, 0] # = get_row(np.dot(revised_A1_inv, revised_A2), p).reshape(-1, 1)
        # revise z
        revised_z = revise_z(z, revised_alpha, w_p, p, q)

        # TODO: REVISE c, c1, c2!
        
        # revision
        x = revised_x.copy()
        N = revised_N.copy()
        N_inv = revised_N_inv.copy()
        A1 = revised_A1.copy()
        A1_inv = revised_A1_inv.copy()
        A2 = revised_A2.copy()
        A = revised_A.copy()
        z = revised_z.copy()

        iteration_number += 1
        if iteration_number == 1_000_000:
            raise Exception("too long computation")
            break

c = [1, 0, 0, 0, -1]
A = pd.DataFrame({'x1': [0, 0, 0], 'x2': [0, 0, 1], 'x3': [0, 1, 0], 'x4': [1, 0, 0], 'x5': [0, 0, 1]})
b = np.array([[0], [1], [2]])
simplex(c, A, b)

[[0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
ITERATIONS: 1


array([0., 0., 1., 0., 2.])

In [14]:
c, A, b = prepare_problem(df.x, df.y, 0.01)
simplex(c, A, b)

[[ 1.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  0. -1.  0.  0.  0.  0.  0.
   0.  0.]
 [ 1.  0.  0.  1.  0.  0.  0.  0. -1.  0.  0.  0.  0. -1.  0.  0.  0.  0.
   0.  0.]
 [ 1.  1.  0.  0.  0.  0.  0.  0. -1. -1.  0.  1.  0.  0. -1.  0.  0.  0.
   0.  0.]
 [ 1.  1.  0.  0.  0.  1.  0.  0. -1. -1.  0.  0.  0.  0.  0. -1.  0.  0.
   0.  0.]
 [ 1.  1.  0.  0.  0.  0.  1.  0. -1. -1.  0.  0.  0.  0.  0.  0. -1.  0.
   0.  0.]
 [ 1.  2.  0.  0.  0.  0.  0.  1. -1. -2.  0.  0.  0.  0.  0.  0.  0. -1.
   0.  0.]
 [ 1.  2.  0.  0.  0.  0.  0.  0. -1. -2.  1.  0.  0.  0.  0.  0.  0.  0.
  -1.  0.]
 [ 1.  3.  0.  0.  1.  0.  0.  0. -1. -3.  0.  0.  0.  0.  0.  0.  0.  0.
   0. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.

array([-2.,  2.,  0.,  0.,  2.,  3.,  0.,  1.,  2., -1.,  0., -1.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [12]:
steepest_edge_simplex(c, A, b)

array([-2.,  2.,  0.,  0.,  2.,  3.,  0.,  1.,  2., -1.,  0., -1.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.])